In [7]:
# Imports
import pandas as pd
import numpy as np
from sqlalchemy import create_engine  
import os
from dotenv import load_dotenv

In [8]:
df = pd.read_csv('dataset/game_sales_data.csv',encoding='ISO-8859-1',index_col='Rank')
# print(df.shape[0])
# print(df.columns)
print(df[['User_Score','Critic_Score']].isnull().sum())
print('Amount of rows where both are null:',df[['User_Score','Critic_Score']].isnull().all(axis=1).sum())


User_Score      17377
Critic_Score     9631
dtype: int64
Amount of rows where both are null: 9616


In [9]:
# Data Preparation (separate the initial csv into two and export them)

df = pd.read_csv('dataset/game_sales_data.csv', encoding='ISO-8859-1',index_col='Rank') # Make sure the index column is recognized (No need to specify it or drop it afterwards)
# display(df)
df = df.rename(columns={'Name': 'game', 'Critic_Score': 'critic_score','Platform': 'platform', 'Publisher': 'publisher', 'Developer': 'developer', 'User_Score': 'user_score','Total_Shipped': 'games_sold', 'Year': 'year'})

game_sales=df[['game','platform','publisher','developer','games_sold','year']]
# display(game_sales)
game_sales.to_csv('dataset/game_sales.csv')
reviews = df[['game','critic_score','user_score']]
# display(reviews)
reviews.to_csv('dataset/reviews.csv')

## 0. Upload the csv to Snowflake
-Log into Snowflake<br>
-Under Data -> Databases -> (top right) + Database <br>
-Select the newly created database and select 'public' schema <br>
&nbsp;&nbsp;&nbsp; OR (top right) + Schema<br>
-Select the schema and (top right) Create -> Table -> From File<br>
-Select the files, write a name and click Next.


In [10]:
# Load data from the .env file

load_dotenv()

snowflake_user = os.getenv("SNOWFLAKE_USER")
snowflake_password = os.getenv("SNOWFLAKE_PASSWORD")
snowflake_account = os.getenv("SNOWFLAKE_ACCOUNT")
snowflake_database = os.getenv("SNOWFLAKE_DATABASE")
snowflake_schema = os.getenv("SNOWFLAKE_SCHEMA")
snowflake_warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")

Snowflake_connect = f"snowflake://{snowflake_user}:{snowflake_password}@{snowflake_account}/{snowflake_database}/{snowflake_schema}?warehouse={snowflake_warehouse}"


# database connection to Snowflake
e = create_engine(Snowflake_connect) 


## 1. Top 10 games

In [12]:
# Select all information for the top ten best-selling games and order the results from best-selling game down to tenth best-selling.

query=''' 
SELECT * 
FROM game_sales 
ORDER BY games_sold DESC 
LIMIT 10;
'''

# Select all information for the top ten best-selling games and order the results from best-selling game down to tenth best-selling. Then, order  those results by year from recent to oldest.

query2='''
SELECT *
FROM (
    SELECT * 
    FROM game_sales 
    ORDER BY games_sold DESC 
    LIMIT 10
)
ORDER BY year DESC;
'''

display(pd.read_sql(query2, con=e))


,rank,game,platform,publisher,developer,games_sold,year
0,5,PLAYERUNKNOWN'S BATTLEGROUNDS,PC,PUBG Corporation,PUBG Corporation,37,2017
1,3,Counter-Strike: Global Offensive,PC,Valve,Valve Corporation,40,2012
2,6,Minecraft,PC,Mojang,Mojang AB,33,2010
3,7,Wii Sports Resort,Wii,Nintendo,Nintendo EAD,33,2009
4,10,New Super Mario Bros. Wii,Wii,Nintendo,Nintendo EAD,30,2009
5,4,Mario Kart Wii,Wii,Nintendo,Nintendo EAD,37,2008
6,1,Wii Sports,Wii,Nintendo,Nintendo EAD,83,2006
7,9,New Super Mario Bros.,DS,Nintendo,Nintendo EAD,31,2006
8,8,Pokemon Red / Green / Blue Version,GB,Nintendo,Game Freak,31,1998
9,2,Super Mario Bros.,NES,Nintendo,Nintendo EAD,40,1985


## 2. Missing review scores
<p>Wow, the best-selling video games were released between 1985 to 2017! That's quite a range; we'll have to use data from the <code>reviews</code> table to gain more insight on the best years for video games. </p>
<p>First, it's important to explore the limitations of our database. One big shortcoming is that there is not any <code>reviews</code> data for some of the games on the <code>game_sales</code> table. </p>

In [13]:
# Join games_sales and reviews and display the count of games where both critic_score and user_score are null.

query='''
SELECT 
    COUNT(*)
FROM game_sales
LEFT JOIN reviews
    ON game_sales.rank = reviews.rank
WHERE critic_score IS NULL AND user_score IS NULL;
'''

display(pd.read_sql(query, con=e))


,COUNT(*)
0,9616


## 3. Years that video game critics loved
<p>We can continue our exploration, while keeping in mind that half of our rows contain missing critic and user score. </p>
<p>There are lots of ways to measure the best years for video games! Let's start with what the critics think. </p>

In [14]:
# Select release year and average critic score for each year, rounded and aliased
# Join the game_sales and reviews tables
# Group by release year
# Order the data from highest to lowest avg_critic_score and limit to 10 results (make sure to exclude any NULL result)

query='''
SELECT
    year,
    ROUND(AVG(critic_score), 2) AS avg_critic_score
FROM reviews
NATURAL JOIN game_sales
GROUP BY year
    HAVING AVG(critic_score) IS NOT NULL
ORDER BY avg_critic_score DESC
LIMIT 10;
'''


'''N.B. By default in Snowflake, using an aggregate function will only consider the values that are not NULL so we don't need to specify a condition in the SELECT. 
However, there are scenarios where all the values in the grouping are NULL, resulting in a NULL aggregate. 
This is why we specify HAVING AVG(critic_score) IS NOT NULL.'''



# Now order those results by year from recent to oldest.
query2='''
SELECT *
FROM(
    SELECT
    year,
    ROUND(AVG(critic_score), 2) AS avg_critic_score
    FROM reviews
    NATURAL JOIN game_sales
    GROUP BY year
        HAVING AVG(critic_score) IS NOT NULL
    ORDER BY avg_critic_score DESC
    LIMIT 10
    )
ORDER BY year DESC;
'''

display(pd.read_sql(query2, con=e))
    

,year,avg_critic_score
0,2020,8.33
1,2019,7.95
2,1994,8.67
3,1993,8.09
4,1992,9.25
5,1991,8.63
6,1990,8.83
7,1984,10.00
8,1982,9.00
9,1981,8.00


## 4. Number of games
<p>The range of great years according to critic reviews goes from 1981 until 2020: we are no closer to finding the golden age of video games! </p>
<p>Hang on, though. Some of those <code>avg_critic_score</code> values look like suspiciously round numbers for averages. The value for 1982 and 1981 look especially fishy. Maybe there weren't a lot of video games in our dataset that were released in certain years. </p>
<p>Let's update our query and find out whether 1982 really was such a great year for video games.</p>

In [15]:
# Update the query to add a count of games released in each year called num_games
# Update the query so that it only returns years that have more than four reviewed games

query='''
SELECT *
FROM(
    SELECT
        year,
        ROUND(AVG(critic_score), 2) AS avg_critic_score,
        COUNT(game) AS num_games
    FROM reviews
    NATURAL JOIN game_sales
    GROUP BY year
        HAVING COUNT(game) > 4 AND AVG(critic_score) IS NOT NULL
    ORDER BY avg_critic_score DESC
    LIMIT 10
    )
ORDER BY year DESC;
'''

display(pd.read_sql(query, con=e))

,year,avg_critic_score,num_games
0,2020,8.33,13
1,2019,7.95,63
2,1994,8.67,104
3,1993,8.09,61
4,1992,9.25,40
5,1991,8.63,34
6,1990,8.83,21
7,1984,10.00,12
8,1982,9.00,47
9,1981,8.00,7


## 5. Years that dropped off the critics' favorites list
<p>That looks better! The <code>num_games</code> column convinces us that our new list of the critics' top games reflects years that had quite a few well-reviewed games rather than just one or two hits. But which years dropped off the list due to having four or fewer reviewed games? Let's identify them so that someday we can track down more game reviews for those years and determine whether they might rightfully be considered as excellent years for video game releases!</p>
<p>It's time to brush off your set theory skills. To get started, we've created tables with the results of our previous two queries:</p>
<h3 id="top_critic_years"><code>top_critic_years</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>

In [16]:
# Select the year and avg_critic_score for those years that dropped off the list of critic favorites 
# Order the results from highest to lowest avg_critic_score

query='''
SELECT
    year,
    avg_critic_score
FROM top_critic_years
WHERE year NOT IN (SELECT year FROM top_critic_years_more_than_four_games)
ORDER BY avg_critic_score DESC;
'''

display(pd.read_sql(query, con=e))

ProgrammingError: (snowflake.connector.errors.ProgrammingError) 002003 (42S02): SQL compilation error:
Object 'TOP_CRITIC_YEARS' does not exist or not authorized.
[SQL: 
SELECT
    year,
    avg_critic_score
FROM top_critic_years
WHERE year NOT IN (SELECT year FROM top_critic_years_more_than_four_games)
ORDER BY avg_critic_score DESC;
]
(Background on this error at: https://sqlalche.me/e/14/f405)

## 6. Years video game players loved
<p>Based on our work in the task above, it looks like the early 1990s might merit consideration as the golden age of video games based on <code>critic_score</code> alone, but we'd need to gather more games and reviews data to do further analysis. </p>
<p>Let's move on to looking at the opinions of another important group of people: players! To begin, let's create a query very similar to the one we used in Task Four, except this one will look at <code>user_score</code> averages by year rather than <code>critic_score</code> averages.</p>

In [ ]:
# Select year, an average of user_score, and a count of games released in a given year, aliased and rounded
# Include only years with more than four reviewed games; group data by year
# Order data by avg_user_score, and limit to ten results

query='''
SELECT
    year,
    ROUND(AVG(user_score), 2) AS avg_user_score,
    COUNT(game) AS num_games
FROM reviews
JOIN game_sales
    USING(game)
GROUP BY year
    HAVING COUNT(game) > 4
ORDER BY avg_user_score DESC
LIMIT 10;
'''

display(pd.read_sql(query, con=e))

## 7. Years that both players and critics loved
<p>Alright, we've got a list of the top ten years according to both critic reviews and user reviews. Are there any years that showed up on both tables? If so, those years would certainly be excellent ones!</p>
<p>Recall that we have access to the <code>top_critic_years_more_than_four_games</code> table, which stores the results of our top critic years query from Task 4:</p>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>
<p>We've also saved the results of our top user years query from the previous task into a table:</p>
<h3 id="top_user_years_more_than_four_games"><code>top_user_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_user_score</code></td>
<td>float</td>
<td>Average of all user scores for games released in that year</td>
</tr>
</tbody>
</table>

In [ ]:
# Select the year results that appear on both tables

query='''
SELECT
    year
FROM top_critic_years_more_than_four_games
NATURAL JOIN top_user_years_more_than_four_games;
'''

# Other solution
query2='''
SELECT
    year
FROM top_critic_years_more_than_four_games
WHERE year IN (
                SELECT 
                    year 
                FROM top_user_years_more_than_four_games
            );
'''

display(pd.read_sql(query, con=e))

## 8. Sales in the best video game years
<p>Looks like we've got three years that both users and critics agreed were in the top ten! There are many other ways of measuring what the best years for video games are, but let's stick with these years for now. We know that critics and players liked these years, but what about video game makers? Were sales good? Let's find out.</p>
<p>This time, we haven't saved the results from the previous task in a table for you. Instead, we'll use the query from the previous task as a subquery in this one! This is a great skill to have, as we don't always have write permissions on the database we are querying.</p>

In [ ]:
# Select year and sum of games_sold, aliased as total_games_sold; order results by total_games_sold descending
# Filter game_sales based on whether each year is in the list returned in the previous task

query='''
SELECT
    year,
    SUM(games_sold) AS total_games_sold
FROM game_sales
WHERE year IN (
                SELECT 
                    year 
                FROM top_critic_years_more_than_four_games
                WHERE year IN (
                    SELECT year 
                    FROM top_user_years_more_than_four_games
                )
            )
GROUP BY year;
'''

display(pd.read_sql(query, con=e))

ProgrammingError: (snowflake.connector.errors.ProgrammingError) 002003 (42S02): SQL compilation error:
Object 'TOP_CRITIC_YEARS_MORE_THAN_FOUR_GAMES' does not exist or not authorized.
[SQL: 
SELECT
    year,
    SUM(games_sold) AS total_games_sold
FROM game_sales
WHERE year IN (SELECT year FROM top_critic_years_more_than_four_games
               WHERE year IN (SELECT year FROM top_user_years_more_than_four_games))
GROUP BY year;
]
(Background on this error at: https://sqlalche.me/e/14/f405)